# Creating a Chatbot linked to the Database

## 1. Importing libraries

In [ ]:
%pip install surrealist openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [22]:
from surrealist import Surreal
from openai import OpenAI
import os

%reload_ext dotenv
%dotenv .env

## 2. Connect to the database

In [23]:
surreal = Surreal("http://127.0.0.1:28900", credentials=(os.getenv("SURREAL_USER"), os.getenv("SURREAL_PASS"))) 
print(surreal.is_ready())
print(surreal.version())

True
surrealdb-2.1.2


## 4. Embed articles and save to database

In [ ]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"
llm_model_name = "gpt-4o-mini"
system_prompt = """
You are an Algerian legal expert AI trained to provide accurate answers.
You must strictly base your answer on the provided context.
You must avoid at all cost to mention in your answer that you are being provided with context.
You must strictly answer in English.
Translate the keywords from context to English.
Your response must adhere to any requested format in the question, if present.
Interpret the context to extract required information and present it directly, without mentioning the reasoning process.
If not, state clearly that you are unable to answer the question.
Do not fabricate or assume facts.
""".strip()

def search_articles(query):
    response = client.embeddings.create(
        input=query,
        model=embed_model_name,
    )
    q_vector = response.data[0].embedding

    with surreal.connect() as db:
        db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))
        res = db.query("SELECT text, vector::similarity::cosine(embed_vector, $q_vector) AS sim FROM articles ORDER BY sim DESC LIMIT 5", {"q_vector": q_vector})

    return res.result

def ask_question(query):
    articles = search_articles(query)

    user_prompt = "```context\n"
    for article in articles:
        user_prompt += f"{article['text']}\n"
    user_prompt += "```\n"
    user_prompt += f"```question\n{query}\n```"

    print(user_prompt)
    print("\n------------------\n")

    stream = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )
    
    for chunk in stream:
        text_chunk = chunk.choices[0].delta.content
        if text_chunk is None:
            continue
        print(text_chunk, end="")

# question = "if I'm christian, what should I do in Algeria? and give me brief answer"
# question = "Anything about family law"
question = "معلومات عن الانتخابات"
ask_question(question)

```context
المادة 193 : تُلزم السلطات العمومية المكلفة بتنظيم الانتخابات بإحاطاتها بالشفافية والحياد. وبهذه الصفة، توضع القائمة الانتخابية عند كل انتخاب تحت تصرف المترشحين. يحدد القانون العضوي المتعلق بنظام الانتخابات كيفيات تطبيق هذا الحكم.
المادة 178 : تضطلع المجالس المنتخَبة بوظيفة الرّقابة في مدلولها الشّعبيّ.
المادة 194 : تُحدث هيئة عليا مستقلة لمراقبة الانتخابات. ترأس الهيئة شخصية وطنية يعينها رئيس الجمهورية، بعد استشارة الأحزاب السياسية. للهيئة العليا لجنة دائمة. وتنشر الهيئة العليا أعضاءها الآخرين فور استدعاء الهيئة الانتخابية. تتكون الهيئة العليا بشكل متساو من : - قضاة يقترحهم المجلس الأعلى للقضاء، ويعينهم رئيس الجمهورية، - وكفاءات مستقلة يتم اختيارها من ضمن المجتمع المدني، يعينها رئيس الجمهورية. تسهر اللجنة العليا على شفافية الانتخابات الرئاسية والتشريعية والمحلية وكذا الاستفتاء ونزاهتها، منذ استدعاء الهيئة الناخبة حتى إعلان النتائج المؤقتة للاقتراع. تسهر اللجنة الدائمة للهيئة العليا على الخصوص على ما يأتي : - الإشراف على عمليات مراجعة الإدارة للقوائم الانتخابية، - صياغة التو